In [1]:

from roboflow import Roboflow
rf = Roboflow(api_key="Insert your API key")
project = rf.workspace("uni-4phrc").project("aml_train")
dataset = project.version(1).download("folder")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: cycler
    Found existing ins


Extracting Dataset Version Zip to aml_train-1 in folder:: 100%|██████████| 4129/4129 [00:00<00:00, 6312.22it/s]


In [2]:
rf = Roboflow(api_key="Insert your API key")
project = rf.workspace("uni-4phrc").project("aml_test")
dataset = project.version(1).download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to aml_test-1 in folder:: 100%|██████████| 4004/4004 [00:00<00:00, 7084.70it/s]


In [245]:
rf = Roboflow(api_key="Insert your API key")
project = rf.workspace("aml-cg1zr").project("birddeep-bla72")
dataset = project.version(1).download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to BirdDeep-1 in folder:: 100%|██████████| 23294/23294 [00:03<00:00, 6577.87it/s]


In [183]:
rf = Roboflow(api_key="Insert your API key")
project = rf.workspace("aml-cg1zr").project("birddeep-bla72")
dataset = project.version(2).download("folder")

loading Roboflow workspace...
loading Roboflow project...
Exporting format folder in progress : 85.0%
Version export complete for folder format



Extracting Dataset Version Zip to BirdDeep-2 in folder:: 100%|██████████| 23294/23294 [00:03<00:00, 6503.52it/s]


In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import datetime
from tensorflow.keras.applications import InceptionV3
import re


In [2]:
train_df = pd.read_csv("train_images.csv")
test_df = pd.read_csv("test_images_path.csv")

train_folder = './BirdDeep-1/train'
test_folder = './aml_test-1/test/test_images'


In [24]:
# Load train data
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_folder,
  validation_split=0.2,
  subset="training",
  labels='inferred',
  class_names=train_df['label'].apply(lambda x: str(x)).unique().tolist(),
  seed=42,
  image_size=(299, 299),
  batch_size=16)

Found 14726 files belonging to 200 classes.
Using 11781 files for training.


In [25]:
# Load validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_folder,
  validation_split=0.2,
  subset="validation",
  labels='inferred',
  class_names=train_df['label'].apply(lambda x: str(x)).unique().tolist(),
  seed=42,
  image_size=(299, 299),
  batch_size=16)

Found 14726 files belonging to 200 classes.
Using 2945 files for validation.


In [26]:
# Load test data paths
test_df = pd.read_csv("test_images_path.csv")
test_images = test_folder = './aml_test-1/test/test_images'

In [27]:
# Load test data
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_folder,
  labels=None,
  seed=42,
  shuffle = False,
  image_size=(299, 299),
  batch_size=1)

Found 4000 files belonging to 1 classes.


In [28]:
#Normalization
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x: (normalization_layer(x)))

In [29]:
# run below if you want to check normalization
image_batch, labels_batch = next(iter(train_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0



**Hyperparameter tuning**

In [9]:
from sklearn.model_selection import ParameterGrid

# Define a function to create and compile the model with given hyperparameters
def create_model(learning_rate, dropout_rate):
    base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
    avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    dropout = Dropout(dropout_rate)(avg)
    output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    return model

# Define a grid of hyperparameters to search through
param_grid = {
    'learning_rate': [0.005, 0.001, 0.0005],
    'dropout_rate': [0.3, 0.5, 0.7]
}

best_val_accuracy = 0
best_params = {}

# Iterate through each combination of hyperparameters
for params in ParameterGrid(param_grid):
    print(f"Training with params: {params}")
    
    # Create and compile the model with current hyperparameters
    model = create_model(params['learning_rate'], params['dropout_rate'])
    
    # Train the model for one epoch
    history = model.fit(
        train_ds,
        epochs=1,
        validation_data=val_ds  # Set verbose to 0 to avoid printing training progress
    )
    
    # Evaluate the model on the validation set
    val_accuracy = history.history['val_accuracy'][0]
    print(f"Validation accuracy: {val_accuracy}")
    
    # Check if this set of hyperparameters is the best so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = params

print(f"Best validation accuracy: {best_val_accuracy}")
print(f"Best hyperparameters: {best_params}")

Training with params: {'dropout_rate': 0.3, 'learning_rate': 0.005}


2023-12-14 22:34:14.430035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2023-12-14 22:34:14.527199: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


47/47 [==============================] - 211s 4s/step - loss: 3.3981 - accuracy: 0.2878 - val_loss: 2.2561 - val_accuracy: 0.4716
Validation accuracy: 0.47164684534072876
Training with params: {'dropout_rate': 0.3, 'learning_rate': 0.001}


2023-12-14 22:37:46.800532: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


47/47 [==============================] - 209s 4s/step - loss: 4.5535 - accuracy: 0.1445 - val_loss: 3.7615 - val_accuracy: 0.3188
Validation accuracy: 0.3188455104827881
Training with params: {'dropout_rate': 0.3, 'learning_rate': 0.0005}


...

47/47 [==============================] - 211s 4s/step - loss: 3.8645 - accuracy: 0.1778 - val_loss: 2.4844 - val_accuracy: 0.4482
Validation accuracy: 0.44821733236312866
Training with params: {'dropout_rate': 0.7, 'learning_rate': 0.001}


2023-12-14 22:59:10.508554: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


47/47 [==============================] - 217s 5s/step - loss: 4.7827 - accuracy: 0.0786 - val_loss: 4.0486 - val_accuracy: 0.3022
Validation accuracy: 0.3022071421146393
Training with params: {'dropout_rate': 0.7, 'learning_rate': 0.0005}


2023-12-14 23:02:46.122777: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


47/47 [==============================] - 212s 4s/step - loss: 5.0727 - accuracy: 0.0354 - val_loss: 4.5889 - val_accuracy: 0.2105
Validation accuracy: 0.21052631735801697
Best validation accuracy: 0.47164684534072876
Best hyperparameters: {'dropout_rate': 0.3, 'learning_rate': 0.005}


## Model

In [31]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras import regularizers
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
dropout = Dropout(0.3)(avg) 
output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

#output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False
    
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [32]:
# Train the model with early stopping
model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[early_stopping]
)

Epoch 1/10


2023-12-14 23:49:10.517479: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


737/737 [==============================] - 187s 248ms/step - loss: 3.2477 - accuracy: 0.3065 - val_loss: 2.2317 - val_accuracy: 0.4944
Epoch 2/10
737/737 [==============================] - 183s 249ms/step - loss: 1.7546 - accuracy: 0.5923 - val_loss: 1.7796 - val_accuracy: 0.5752
Epoch 3/10
737/737 [==============================] - 182s 247ms/step - loss: 1.2943 - accuracy: 0.7023 - val_loss: 1.5654 - val_accuracy: 0.6214
Epoch 4/10
737/737 [==============================] - 183s 248ms/step - loss: 1.0181 - accuracy: 0.7701 - val_loss: 1.4569 - val_accuracy: 0.6387
Epoch 5/10
737/737 [==============================] - 183s 248ms/step - loss: 0.8330 - accuracy: 0.8147 - val_loss: 1.3698 - val_accuracy: 0.6570
Epoch 6/10
737/737 [==============================] - 182s 247ms/step - loss: 0.6956 - accuracy: 0.8500 - val_loss: 1.2944 - val_accuracy: 0.6788
Epoch 7/10
737/737 [==============================] - 183s 249ms/step - loss: 0.5860 - accuracy: 0.8784 - val_loss: 1.2539 - val_accura

In [33]:
def extract_first_number_underscore(string_list):
    pattern = r'(\d+)_\w+'  # Capture the first sequence of digits before the first underscore
    numbers = [re.search(pattern, s).group(1) for s in string_list if re.search(pattern, s)]
    return numbers

def extract_first_number_points(string_list):
    pattern = r'(\d+).\w+'  # Capture the first sequence of digits before the first underscore
    numbers = [re.search(pattern, s).group(1) for s in string_list if re.search(pattern, s)]
    return numbers

In [35]:
dir_folder = sorted(os.listdir(test_folder))
dir_folder_numbers = extract_first_number_underscore(dir_folder)
test_path_numbers = extract_first_number_points(test_df.image_path.values.tolist())
test_df['path_number'] = test_path_numbers
# Convert the 'Values' column to Categorical with custom ordering
test_df['path_number'] = pd.Categorical(test_df['path_number'], categories=dir_folder_numbers, ordered=True)

# Sort the DataFrame based on the 'Values' column
test_df = test_df.sort_values(by='path_number')

# Reset the index if needed
# test = df_sorted.reset_index(drop=True)

test_df

,id,image_path,label,path_number
3996,3997,/test_images/1000.jpg,1,1000
3995,3996,/test_images/1001.jpg,1,1001
3994,3995,/test_images/1002.jpg,1,1002
3993,3994,/test_images/1003.jpg,1,1003
3992,3993,/test_images/1004.jpg,1,1004
...,...,...,...,...
2,3,/test_images/997.jpg,1,997
1,2,/test_images/998.jpg,1,998
0,1,/test_images/999.jpg,1,999
10,11,/test_images/99.jpg,1,99


In [36]:
predictions = model.predict(test_ds)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) +1

# Update the 'label' column in the test dataframe with the predicted labels
test_df['label'] = predicted_labels



4000/4000 [==============================] - 79s 19ms/step


In [39]:
selected_columns = ['id', 'label']
test_df_selected = test_df[selected_columns]
script_dir = os.getcwd()

In [40]:
test_df_selected = test_df_selected.sort_values(by = 'id')

# Generate a timestamp for the filename
current_datetime = datetime.datetime.now()
timestamp = current_datetime.strftime("%Y%m%d_%H%M")
file_name = f"submission_{timestamp}.csv"

# Define the folder for submissions
submissions_folder = os.path.join(script_dir, 'sub')

# Specify the full file path
file_path = os.path.join(submissions_folder, file_name)

# Save the final submission file
test_df_selected.to_csv(file_path, index=False)